# Ordinal Regression with Community Innovation Survey Data

In [26]:
library(data.table)
library(magrittr)
library(knitr)
library(ggplot2)
library(corrplot)
library(oglmx)
library(MASS)
library(stargazer)


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




In [5]:
# read data
file_path <- "../data/innovation_survey/extmidp21.csv"
survey <- fread(file_path)
str(survey)

Classes ‘data.table’ and 'data.frame':	5083 obs. of  284 variables:
 $ id          : int  300127 301003 301078 301084 301189 301282 301396 301420 301591 301948 ...
 $ branche     : chr  "Elektroindustrie" "Metallerzeugung/-bearbeitung" "Maschinenbau" "Energie/Bergbau/Mineraloel" ...
 $ bran_4      : chr  "Forschungsintensive Industrie" "Sonstige Industrie" "Forschungsintensive Industrie" "Sonstige Industrie" ...
 $ filter      : chr  "nein" "ja" "nein" "ja" ...
 $ ost         : chr  "ost" "ost" "west" "west" ...
 $ ustaat      : chr  "" "" "" "" ...
 $ gb          : chr  "Bereich" "Bereich" "Bereich" "Bereich" ...
 $ bges        : num  38.4 4.05 497.85 311.48 751.19 ...
 $ gk3n        : chr  "50-249Besch" "<50Besch" ">=250Besch" "50-249Besch" ...
 $ bges18      : num  38.4 5.06 NA 290.13 NA ...
 $ gk3n18      : chr  "50-249Besch" "<50Besch" "" "50-249Besch" ...
 $ bges19      : num  38.4 4.05 NA 297.04 NA ...
 $ gk3n19      : chr  "50-249Besch" "<50Besch" "" "50-249Besch" ...
 $ bhsp  

In [12]:

# our dependent variables
#oekpz1 - Reduced energy use per unit of output


# our independent variables
#oekfak1 - Existing environmental regulations
#oekfak8 - Increasing cost of energy,  water or material

# formula for ordinal logistic regression
formula1 <- ordered(oekpz1, levels=c("nein", "ja, gering", "ja, bedeutend")) ~ oekfak1 + oekfak8 + branche

model1 <- ologit.reg(formula1, data=survey)

Warning message in model.response(mf, "numeric"):
“using type = "numeric" with a factor response will be ignored”


In [13]:
summary(model1)

Ordered Logit Regression 
Log-Likelihood: -3462.441 
No. Iterations: 5 
McFadden's R2: 0.1358867 
AIC: 6984.882 
                                               Estimate Std. error t value
oekfak1gering                                  0.819906   0.339423  2.4156
oekfak1hoch                                    1.235451   0.337314  3.6626
oekfak1mittel                                  1.048016   0.336526  3.1142
oekfak1nein                                    0.290977   0.338702  0.8591
oekfak8gering                                  0.277433   0.315570  0.8792
oekfak8hoch                                    0.824741   0.313099  2.6341
oekfak8mittel                                  0.549970   0.312481  1.7600
oekfak8nein                                   -0.860842   0.318773 -2.7005
brancheEDV/Telekommunikation                  -0.432344   0.240801 -1.7954
brancheElektroindustrie                       -0.269695   0.215562 -1.2511
brancheEnergie/Bergbau/Mineraloel             -0.400116   0.23

In [36]:
# ja, gering means yes but low
# ja, bedeutend means yes but high
margins.oglmx(model1)

Marginal Effects on Pr(Outcome==nein)
                                              Marg. Eff Std. error t value
oekfak1gering                                 -0.199820   0.083000 -2.4075
oekfak1hoch                                   -0.298350   0.078300 -3.8103
oekfak1mittel                                 -0.254785   0.080287 -3.1734
oekfak1nein                                   -0.068396   0.079630 -0.8589
oekfak8gering                                 -0.066412   0.076775 -0.8650
oekfak8hoch                                   -0.200201   0.076321 -2.6231
oekfak8mittel                                 -0.132640   0.076518 -1.7334
oekfak8nein                                    0.194641   0.068451  2.8435
brancheEDV/Telekommunikation                   0.095502   0.049424  1.9323
brancheElektroindustrie                        0.061161   0.047018  1.3008
brancheEnergie/Bergbau/Mineraloel              0.088792   0.048349  1.8365
brancheFahrzeugbau                             0.017209   0.05

In [46]:
# only print out ja, bedeutend
margins.oglmx(model1)[3]

,Marg. Eff,Std. error,t value,Pr(>|t|)
oekfak1gering,0.072687611,0.037722613,1.9268976,5.399240e-02
oekfak1hoch,0.119801581,0.043880736,2.7301634,6.330293e-03
oekfak1mittel,0.098755811,0.041634780,2.3719546,1.769426e-02
oekfak1nein,0.020351758,0.024056750,0.8459895,3.975586e-01
oekfak8gering,0.020784123,0.025610902,0.8115342,4.170590e-01
oekfak8hoch,0.071186459,0.033271295,2.1395758,3.238906e-02
oekfak8mittel,0.043622058,0.028359661,1.5381728,1.240064e-01
oekfak8nein,-0.055815465,0.019816660,-2.8165929,4.853600e-03
brancheEDV/Telekommunikation,-0.025297803,0.011844360,-2.1358522,3.269146e-02
brancheElektroindustrie,-0.016829350,0.012124913,-1.3879976,1.651378e-01
